Victor Mikhaylov, vmikhayl@ias.edu<br>Institute for Advanced Study, 2021-2023

# Modeling structures (or just running seqnn)

In [1]:
import os,stat
import pickle
import pandas as pd

#import the scripts that will create AlphaFold inputs and parse the outputs:
from tfold.modeling import make_inputs,result_parse_tools

MHC loading from MHC.pckl. To update the pickle file, set use_pickle to False
loaded 26122 MHC sequences in  1.6 s


2023-03-05 21:00:01.751413: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


The input should be a pandas dataframe with columns `pep` (peptide sequence) and `MHC allele` or `MHC sequence`.
If both `MHC allele` and `MHC sequence` are provided, the allele will be ignored and the sequence will be used in modeling. MHC class will be imputed from the allele or sequence, and both class I and class II pMHCs can be present in a single run.

`MHC allele` format:<br>
class I: `SpeciesId-Locus*Allele`,<br>
class II: `SpeciesId-LocusA*AlleleA/LocusB*AlleleB`.<br>
Here `SpeciesId` can be `HLA` for human, `H2` for mouse, or NCBI taxonomy ID for other species, e.g. `9796` for horse. (For human and mouse, you can also use NCBI Ids `9606` and `10090` instead of `HLA` and `H2`.) <br>
Some examples of allele notations:<br>
class I:<br>
human: `HLA-A*02:01`, `HLA-B*08:01`, `HLA-C*01:01`;<br>
mouse: `H2-D*p`, `H2-K*d`, `H2-L*q`; <br>
horse: `9796-2*001:01`;<br>
class II:<br>
human: `HLA-DRA*01:01/DRB4*01:144`, `HLA-DPA1*04:01/DPB1*04:01`, `HLA-DQA1*06:01/DQB1*02:05`;<br>
mouse: `H2-IAA*d/IAB*d`, `H2-IEA*d/IEB*k`;<br>
horse: `9796-DRA*001:03/DRB2*003:01`.

Dataframe `make_inputs.seq_tools.mhcs_df` lists all the available MHC alleles:

In [2]:
print(make_inputs.seq_tools.mhcs_df.sample(20))

      species_id  cl chain locus  allele
3620        9606   I     A     A   33:65
11890       9606   I     A     C  07:855
7875        9606   I     A     B  44:506
2232        9606   I     A     A  24:115
3349        9606   I     A     A  31:125
12999       9606   I     A     C  16:125
5516        9606   I     A     B  15:491
24509      60710   I     A     B   14:02
18150       9606  II     B  DPB1  668:01
7546        9606   I     A     B   44:62
260         9606   I     A     A  01:289
24494      60710   I     A     B   11:01
18285       9606  II     B  DPB1  823:01
13941       9606  II     B  DRB1   07:63
20319       9541   I     A    A4   01:05
15964       9606  II     A  DQA1   02:11
24350       9555   I     A     A   23:01
22991       9555   I     A     A   11:01
5863        9606   I     A     B   27:41
3333        9606   I     A     A  31:109


`MHC sequence` must be a string. For class I, it should be the sequence of the alpha-chain (the whole chain or just the antigen-binding region), and for class II, alpha- and beta-chain sequences separated by '/'. When modeling structures from PDB files, and in particular, the datasets used in this work, it is recommended to use `MHC sequence` input because in some PDB files MHC sequences have mutations relative to the alleles that our alignment tool has assigned. (However, Step 1 below takes longer with sequences than with alleles.)

Optional columns in the input dataframe:<br>
- `pmhc_id`: will be used to name folders with models. If not provided, will use 0,1,...<br>
- `pdb_id`: PDB ID for the true structure, including id of the complex within asymmetric unit, e.g. `4zut_0`. If provided, the algorithm will compute peptide and MHC RMSD for each model. Must be: 1) a string; 2) lower case; 3) present in our database; 4) id of complex within asymmetric unit is almost always `0`. You can look at which structures are available in `data/experimental_structures/processed_updated/pdb_rotated`.<br>
- `exclude_pdbs`: list of PDB IDs that should not be used as templates for a given pMHC. Must be a list of lower-case PDB IDs, e.g. `['3gsw','1o5k']`.

As an example, we model one class I and one class II structure from `data/examples/sample.csv`:

In [3]:
df_to_model=pd.read_csv('./data/examples/sample.csv')
print(df_to_model)

               pep                MHC allele  pdb_id
0        NYNYLYRLF               HLA-A*24:02  7f4w_0
1  GIAGFKGEQGPKGEP  HLA-DRA*01:01/DRB1*04:01  6nix_0


<b>Step 1:</b> add `pmhc_id` if not present, add columns `mhc_a` and `mhc_b` with numbered MHC sequences, run `seqnn`

In [4]:
df_to_model=make_inputs.preprocess_df(df_to_model)

#optionally, can use seqnn-f instead of seqnn for class II:
#df_to_model=make_inputs.preprocess_df(df_to_model,use_seqnnf=True)
#(we didn't use it for register filtering in the paper)

#(please ignore warnings from tensorflow)

Retrieving MHC objects from alleles.
making Kd predictions for 1 pmhcs...


2023-03-05 21:00:17.626132: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64:/usr/local/lib64:/usr/local/share/ge2011.11/lib/linux-x64:/usr/local/root/lib:/usr/local/intel/compilers_and_libraries_2019.2.187/linux/compiler/lib/intel64_lin:/usr/local/intel//compilers_and_libraries_2019.2.187/linux/mpi/intel64/libfabric/lib:/usr/local/intel//compilers_and_libraries_2019.2.187/linux/mpi/intel64/lib/release:/usr/local/intel//compilers_and_libraries_2019.2.187/linux/mpi/intel64/lib:/usr/local/intel/compilers_and_libraries_2019.2.187/linux/ipp/lib/intel64:/usr/local/intel/compilers_and_libraries_2019.2.187/linux/compiler/lib/intel64_lin:/usr/local/intel/compilers_and_libraries_2019.2.187/linux/mkl/lib/intel64_lin:/usr/local/intel/compilers_and_libraries_2019.2.187/linux/tbb/lib/intel64/gcc4.7:/usr/local/intel/compilers_and_

making Kd predictions for 1 pmhcs...


/home/vmikhayl/anaconda3/envs/tfold-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/functional.py:588: UserWarning: Input dict contained keys ['regmask'] which did not match any model input. They will be ignored by the model.
  warnings.warn(


`seqnn` predictions are now in columns
- `seqnn_logkds_all`: registers (n_l, n_r) and log_10(K_d) predictions for them;
- `seqnn_logkd`: minimal log_10(K_d);
- `seqnn_tails`: register with the minimal log_10(K_d).

In [5]:
print(df_to_model.columns)

Index(['pep', 'MHC allele', 'pdb_id', 'pmhc_id', 'mhc_a', 'mhc_b', 'class',
       'seqnn_logkds_all', 'seqnn_logkd', 'seqnn_tails'],
      dtype='object')


<b>Step 2:</b> choose folder to store data, prepare and save AlphaFold inputs

In [6]:
#make AF inputs
af_inputs=make_inputs.make_inputs(df_to_model,date_cutoff='2018-04-30',print_stats=True)
#(optionally, set template date cutoff YYYY-MM-DD)
print('total AF runs:',len(af_inputs))

#choose where to put inputs and outputs
working_dir=os.getcwd()+'/tmp/modeling'  #just an example

input_dir=working_dir+'/inputs'
output_dir=working_dir+'/outputs'
os.makedirs(working_dir,exist_ok=True)
os.makedirs(input_dir,exist_ok=True) 
os.makedirs(output_dir,exist_ok=True)

#save AF inputs and input dataframe
#(instead of saving them all in one file, you can split them to then run parallel AF jobs on multiple GPUs)
with open(input_dir+'/input.pckl','wb') as f: 
    pickle.dump(af_inputs,f) 
df_to_model.to_pickle(working_dir+'/target_df.pckl') #will be used by script that parses results
    
#make bash script that will run AlphaFold
af_patch_path=os.getcwd()+'/tfold_run_alphafold.py'
lines=[]
lines.append(f'python {af_patch_path} --inputs {input_dir}/input.pckl --output_dir {output_dir}')
sh_path=working_dir+'/run_alphafold.sh'
with open(sh_path,'w') as f:
    f.writelines('\n'.join(lines))
os.chmod(sh_path, stat.S_IRWXU | stat.S_IXGRP | stat.S_IXOTH)   

class I:
pmhcs:   1; runs:    5, runs per pmhc: av 5.0, max  5; registers per pmhc: av 1.0, max  1
class II:
pmhcs:   1; runs:   10, runs per pmhc: av 10.0, max 10; registers per pmhc: av 2.0, max  2
total AF runs: 15


<b>Step 3:</b> run script `$working_dir/run_alphafold.sh` on a GPU. On NVIDIA A100, it takes ~30 s per model (with 6 and 21 models on average per class I and class II pMHC), plus about 1 min of model compilation which doesn't matter for a large enough number of models. <b>Note</b>: for other parts of the pipeline, running them on GPU gives no advantage.

<b>Step 4:</b> collect and analyze information about the models.

In [7]:
result_parse_tools.parse_results(working_dir)
#saves a summary of models in $working_dir/result_df.pckl

  15 outputs collected in    0.2 s


In [8]:
#info about all models
result_df=pd.read_pickle(working_dir+'/result_df.pckl')

#reduce to best models (lowest predicted score) for each pMHC
best_model_df=result_parse_tools.reduce_to_best(result_df,['pmhc_id'],'score',how='min')

Dataframe best_model_df contains information about best models for each pMHC. It keeps all columns from the input dataframe and adds some more:
- `pmhc_id`;
- `pep`;
- `MHC allele`;
- `model_id`: id of the best model; the model is `$working_dir/$pmhc_id/structure_model_1_$model_id.pdb`;
- `pep_CA`, `mhc_CA`, `pep_all`, `mhc_all`: peptide and MHC C-alpha and all-atom RMSD (if true structure given);
- `af_tails`: predicted register (n_l,n_r);
- `register_identified`: if the model is so bad that register cannot be identified, this will be `False`;
- `score`: 100-pLDDT averaged over the peptide.

In [10]:
best_model_df[['pmhc_id','model_id','pep','MHC allele',
               'pep_CA','pep_all','mhc_CA','mhc_all',
               'af_tails','register_identified',
               'score']]

,pmhc_id,model_id,pep,MHC allele,pep_CA,pep_all,mhc_CA,mhc_all,af_tails,register_identified,score
pmhc_id,,,,,,,,,,,
0,0,0,NYNYLYRLF,HLA-A*24:02,0.595139,1.257157,0.487035,1.252594,"(0, 0)",True,4.864736
1,1,5,GIAGFKGEQGPKGEP,HLA-DRA*01:01/DRB1*04:01,0.376919,1.085748,0.331121,1.059522,"(4, 2)",True,3.768176


# pMHC and TCR structure data and some tools to work with it

## `seq_tools`

`seq_tools` is a simple toolbox for working with numbered sequences. In particular, it describes a class `NUMSEQ` that is used throughout the project.

In [19]:
from tfold.utils import seq_tools
help(seq_tools.NUMSEQ)

Help on class NUMSEQ in module tfold.utils.seq_tools:

class NUMSEQ(builtins.object)
 |  NUMSEQ(**args)
 |  
 |  CONTAINS:
 |  - structured array self.data of dtype numseq_dtype; 
 |  (can add more fields optionally, but then not guaranteed that join_NUMSEQ and other functions will work);    
 |  - a dict self.info to store e.g. species, locus, allele, etc; 
 |  includes at least 'gaps', which is a structured array of numseq_dtype;
 |  INIT TAKES: kwd arguments; arg 'info' becomes self.info, other args go into data;
 |  if arg 'data' given, goes into self.data, otherwise self.data built from args 'seq','num','ins'/'pdbnum'
 |  and others
 |  
 |  Methods defined here:
 |  
 |  __init__(self, **args)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  copy(self)
 |  
 |  count_mutations(self)
 |      counts mutations, gaps, gaps_left, gaps_right
 |  
 |  dump(self)
 |      returns {'data':data,'info':info}. For pickling
 |  
 |  get_fragment(self, num_l, num_

Let's create a NUMSEQ object with a given sequence, numbering, and insertion codes

In [20]:
#creating a NUMSEQ
seq='GILGFVFTL'
num=[0,0,3,4,5,6,7,8,9]
ins=['A','B']+['']*7

seq_obj=seq_tools.NUMSEQ(seq=seq,num=num,ins=ins)

The object contains two data attributes: `seq_obj.data` and `seq_ob.info`. The first is a structured `np.array` containing the sequence, numbering, insertion codes, as well as pdbnum, which are 5-character strings containing both the number and the insertion code of a residue, formatted as characters 23-27 in a pdb ATOM record. The `data` array also contains an array `ss` with secondary structure information and other labels (such as CDR1-3), as well as an array `mutations` that stores mutations relative to a parent sequence, when given

In [21]:
#.data
print(seq_obj.data.dtype) #NUMSEQ.data structured array
print(seq_obj.data[0])    #a slice of the array: aa name, number, insertion code, pdbnum, secondary structure, mutation
print(seq_obj.seq())      #a method for returning the sequence as a string

[('seq', '<U3'), ('num', '<i2'), ('ins', '<U1'), ('pdbnum', '<U5'), ('ss', '<U15'), ('mutations', '<U3')]
('G', 0, 'A', '   0A', '', '')
GILGFVFTL


The `info` attribute is a dictionary where we store things like locus and allele. By default, it also contains an array `gaps` where we can store information about missing residues. 

In [22]:
#.info and gaps
print('dictionary ".info":')
print(seq_obj.info)
seq_obj_with_gap=seq_obj.copy()       #make a copy where we will introduce a gap
seq_obj_with_gap.data[0]['seq']='g'   #replace one aa with a small letter signifying that it is a gap
print('we denote missing residues by small letters:')
print(seq_obj_with_gap.seq())
seq_obj_with_gap=seq_obj_with_gap.ungap_small() #remove small letters in the sequence and save them in .info['gaps']
print('small letter gaps removed:')
print(seq_obj_with_gap.seq())
print('they were moved to the "gap" field in ".info":')
print(seq_obj_with_gap.info)

dictionary ".info":
{'gaps': array([],
      dtype=[('seq', '<U3'), ('num', '<i2'), ('ins', '<U1'), ('pdbnum', '<U5'), ('ss', '<U15'), ('mutations', '<U3')])}
we denote missing residues by small letters:
gILGFVFTL
small letter gaps removed:
ILGFVFTL
they were moved to the "gap" field in ".info":
{'gaps': array([('G', 0, 'A', '   0A', '', '')],
      dtype=[('seq', '<U3'), ('num', '<i2'), ('ins', '<U1'), ('pdbnum', '<U5'), ('ss', '<U15'), ('mutations', '<U3')])}


Pickle and unpickle. To pickle a NUMSEQ object, we convert it to a dict {'data':data,'info':info}. __In pickled data for the structure database, all sequence information is stored in the form of such dictionaries.__ A NUMSEQ object can be easily restored, if one needs its methods.

In [23]:
seq_obj_dict=seq_obj.dump()      #dump NUMSEQ to dict
for k,v in seq_obj_dict.items():
    print(k,v)
print()
seq_obj_restored=seq_tools.load_NUMSEQ(seq_obj_dict) #restore NUMSEQ from dict
print('restored:')
print('data:',seq_obj_restored.data)
print('info:',seq_obj_restored.info)

data [('G', 0, 'A', '   0A', '', '') ('I', 0, 'B', '   0B', '', '')
 ('L', 3, ' ', '   3 ', '', '') ('G', 4, ' ', '   4 ', '', '')
 ('F', 5, ' ', '   5 ', '', '') ('V', 6, ' ', '   6 ', '', '')
 ('F', 7, ' ', '   7 ', '', '') ('T', 8, ' ', '   8 ', '', '')
 ('L', 9, ' ', '   9 ', '', '')]
info {'gaps': array([],
      dtype=[('seq', '<U3'), ('num', '<i2'), ('ins', '<U1'), ('pdbnum', '<U5'), ('ss', '<U15'), ('mutations', '<U3')])}

restored:
data: [('G', 0, 'A', '   0A', '', '') ('I', 0, 'B', '   0B', '', '')
 ('L', 3, ' ', '   3 ', '', '') ('G', 4, ' ', '   4 ', '', '')
 ('F', 5, ' ', '   5 ', '', '') ('V', 6, ' ', '   6 ', '', '')
 ('F', 7, ' ', '   7 ', '', '') ('T', 8, ' ', '   8 ', '', '')
 ('L', 9, ' ', '   9 ', '', '')]
info: {'gaps': array([],
      dtype=[('seq', '<U3'), ('num', '<i2'), ('ins', '<U1'), ('pdbnum', '<U5'), ('ss', '<U15'), ('mutations', '<U3')])}


`seq_tools` also contains MHC and TCR germline protein sequences with IMGT numbering

In [24]:
seq_tools.load_mhcs() #load MHC sequences
seq_tools.load_tcrs() #load TCR sequences

MHC loading from MHC.pckl. To update the pickle file, set use_pickle to False
loaded 26122 MHC sequences in  1.8 s
TCR loading from TCR.pckl. To update the pickle file, set use_pickle to False
loaded 2835 TCR sequences in  0.2 s


These data are available for multiple species. The species information is collected in a dictionary `seq_tools.species`. The keys are NCBI taxonomy ids https://www.ncbi.nlm.nih.gov/taxonomy. Note: species ids are stored as strings, not integers!  The ids for human and mouse are '9606' and '10090', respectively

In [25]:
seq_tools.species

{'37293': ['aona', '', "Nancy Ma's Night Monkey"],
 '57175': ['aoni', '', 'Black-headed Night Monkey'],
 '9505': ['aotr', '', 'Three-striped Night Monkey'],
 '9507': ['atbe', '', 'White-fronted Spider Monkey'],
 '9483': ['caja', '', 'Common Marmoset'],
 '9523': ['camo', '', 'Red-bellied Titi'],
 '9515': ['ceap', '', 'Tufted Capuchin'],
 '9493': ['capy', '', 'Pygmy Marmoset'],
 '9534': ['chae', '', 'Grivet'],
 '9593': ['gogo', '', 'Gorilla'],
 '9580': ['hyla', '', 'Lar Gibbon'],
 '30588': ['lero', '', 'Golden Lion Tamarin'],
 '9540': ['maar', '', 'Stump-tailed Macaque'],
 '9541': ['mafa', '', 'Crab-eating Macaque'],
 '9544': ['mamu', '', 'rhesus monkey'],
 '9545': ['mane', '', 'Southern Pig-tailed Macaque'],
 '54601': ['masi', '', 'Lion-tailed Macaque'],
 '9555': ['paan', '', 'Olive Baboon'],
 '9556': ['pacy', '', 'Yellow Baboon'],
 '9557': ['paha', '', 'Hamadryas Baboon'],
 '9597': ['papa', '', 'Bonobo'],
 '9598': ['patr', '', 'Common Chimpanzee'],
 '43777': ['pipi', '', 'White-faced S

The information about all available MHC and TCR loci and alleles is collected in the dataframes:

In [26]:
print('MHCs:')
print(seq_tools.mhcs_df.head(5))
print('TCRs:')
print(seq_tools.tcrs_df.head(5))

MHCs:
  species_id cl chain locus allele
0       9606  I     A     A  01:01
1       9606  I     A     A  01:02
2       9606  I     A     A  01:03
3       9606  I     A     A  01:06
4       9606  I     A     A  01:07
TCRs:
  species_id chain reg   locus allele
0      37293     A   J  TRAJ15     01
1      37293     A   J  TRAJ15     02
2      37293     A   J   TRAJ2     01
3      37293     A   J  TRAJ22     01
4      37293     A   J  TRAJ23     01


Human MHC alleles are labeled by the 4-digit system. (More precisely, `xx:xx` or `xx:xxx`, i.e. 4 of 5 digits.) Note that different 4-digit alleles can have identical sequences within the antigen-binding region.

All sequences are stored as NUMSEQ objects. Here is an MHC example:

In [27]:
#MHCs
# the keys are (ncbi_species_id_as_str, locus, allele)

mhc_A0201=seq_tools.mhcs['9606','A','02:01']  #human HLA-A*02:01
print(type(mhc_A0201))
print(mhc_A0201.seq())
print(mhc_A0201.info)
print('secondary structure:')
print(mhc_A0201.data['ss'])
print('IMGT numbering:')
print(mhc_A0201.data['pdbnum'])

<class 'tfold.utils.seq_tools.NUMSEQ'>
GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRMEPRAPWIEQEGPEYWDGETRKVKAHSQTHRVDLGTLRGYYNQSEAGSHTVQRMYGCDVGSDWRFLRGYHQYAYDGKDYIALKEDLRSWTAADMAAQTTKHKWEAAHVAEQLRAYLEGTCVEWLRRYLENGKETLQRT
{'species': '9606', 'class': 'I', 'chain': 'A', 'locus': 'A', 'allele': '02:01', 'gaps': array([],
      dtype=[('seq', '<U3'), ('num', '<i2'), ('ins', '<U1'), ('pdbnum', '<U5'), ('ss', '<U15'), ('mutations', '<U3')])}
secondary structure:
['b1' 'b1' 'b1' 'b1' 'b1' 'b1' 'b1' 'b1' 'b1' 'b1' 'b1' 'b1' 'b1' 'b1' 'l'
 'l' 'l' 'b2' 'b2' 'b2' 'b2' 'b2' 'b2' 'b2' 'b2' 'b2' 'b2' 'b2' 'l' 'l'
 'b3' 'b3' 'b3' 'b3' 'b3' 'b3' 'b3' 'b3' 'l' 'l' 'l' 'b4' 'b4' 'b4' 'b4'
 'b4' 'b4' 'b4' 'b4' 'a' 'a' 'a' 'a' 'a' 'a' 'a' 'a' 'a' 'a' 'a' 'a' 'a'
 'a' 'a' 'a' 'a' 'a' 'a' 'a' 'a' 'a' 'a' 'a' 'a' 'a' 'a' 'a' 'a' 'a' 'a'
 'a' 'a' 'a' 'a' 'a' 'a' 'a' 'a' 'a' 'a' 'b1' 'b1' 'b1' 'b1' 'b1' 'b1'
 'b1' 'b1' 'b1' 'b1' 'b1' 'b1' 'b1' 'b1' 'l' 'l' 'l' 'b2' 'b2' 'b2' 'b2'
 'b2' 'b2' 'b2' 'b2' 'b2' 'b

My version of MHC IMGT numbering is as follows. The antigen-binding domain of an MHC protein, both class I and class II, consists of two halves that are homologous to each other. In class I, they are exons 2 and 3 (within the same protein chain), and in class II, they are parts of chains A and B. IMGT defines a numbering for a single exon, i.e. one half of class I antigen-binding domain or one chain in class II. I use this numbering, but add 1000 for exon 3 in class I to distinguish it from exon 2. I also add 1000 to all numbers in chain B in class II.

In [28]:
# TCRs
tcr_TRAVDV=seq_tools.tcrs['9606','TRAV14/DV4','01']
print(type(tcr_TRAVDV))
print(tcr_TRAVDV.seq())
print(tcr_TRAVDV.info)
print('secondary structure:')
print(tcr_TRAVDV.data['ss'])
print('IMGT numbering:')
print(tcr_TRAVDV.data['pdbnum'])

<class 'tfold.utils.seq_tools.NUMSEQ'>
AQKITQTQPGMFVQEKEAVTLDCTYDTSDPSYGLFWYKQPSSGEMIFLIYQGSYDQQNATEGRYSLNFQKARKSANLVISASQLGDSAMYFCAMRE
{'species': '9606', 'chain': 'A/D', 'reg': 'V', 'locus': 'TRAV14/DV4', 'allele': '01', 'gaps': array([],
      dtype=[('seq', '<U3'), ('num', '<i2'), ('ins', '<U1'), ('pdbnum', '<U5'), ('ss', '<U15'), ('mutations', '<U3')])}
secondary structure:
['A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'CDR1' 'CDR1' 'CDR1' 'CDR1' 'CDR1' 'CDR1'
 'CDR1' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C1' 'C1' 'C1' 'C1' 'C1' 'C1' 'C1'
 'C1' 'C1' 'CDR2' 'CDR2' 'CDR2' 'CDR2' 'CDR2' 'CDR2' 'CDR2' 'CDR2' 'C2'
 'C2' 'C2' 'C2' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'E' 'E' 'E' 'E'
 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'F' 'F' 'F' 'F' 'F' 'F' 'F' 'F' 'CDR3'
 'CDR3' 'CDR3' 'CDR3']
IMGT numbering:
['   1 ' '   2 ' '   3 ' '   4 ' '   5 ' '   6 ' '   7 ' '   8 ' '   9 '
 '  10 ' '  11 ' '  12 ' '  13 ' '  14 ' '  15 ' '  16 ' '  17 ' ' 

The toolsbox `seq_tools` also contains a wrapper for running BLASTP search against TCR, MHC and beta2-m databases, a tool for identifying V and J regions and corresponding alleles in a TCR, making a TCR NUMSEQ object from V, J and CDR3, etc

The MHC sequences are from https://www.ebi.ac.uk/ipd/imgt/hla/ for humans, https://www.ebi.ac.uk/ipd/mhc/ for species other than humans and mice, and manually curated from UNIPROT for mice. The TCR sequences are from https://www.imgt.org/download/V-QUEST/.

## `pdb_tools`

A toolbox for dealing with protein structure files. Based on Bio.PDB

In [29]:
from tfold.utils import pdb_tools

Read a structure from a pdb file

In [16]:
filename=#please download 1ao7 and set path to it   ###'./data/experimental_structures/pdb_source/1ao7.pdb' 
structure,header=pdb_tools.parse_pdb(filename)
print(type(structure))
print(type(header),header.keys())

#turn the Bio.PDB model object into a dict, which is sometimes more convenient to use
structure_dict=pdb_tools.get_structure_dict(structure,include_coords=True,keep_hetero=True)
#structure_dict is three nested dicts: 
#chain: pdbnum: atom_name: atom_coordinates
print(structure_dict['A'][' 273 ']['CA'])

<class 'Bio.PDB.Model.Model'>
<class 'dict'> dict_keys(['name', 'head', 'idcode', 'deposition_date', 'release_date', 'structure_method', 'resolution', 'structure_reference', 'journal_reference', 'author', 'compound', 'source', 'has_missing_residues', 'missing_residues', 'keywords', 'journal'])
[ 65.633 -14.979  43.508]


Superimpose two structures. There are two ways to define which residues should be superimposed to which:
- by resmap. A residue map is a list of pairs of chain+pdbnums. E.g. resmap `[['P   1 ','P   5 '],['M    1 ','N 100A']]` means that in structures one and two, respectively, residues 1 and 5 in chain P as well as residues 1 in chain M and 100A in chain N should be superimposed.
- by chainmap. A chain map is a list of pairs of chain letters. Residues with matching pdbnums in the two chains will be superimposed. E.g. chainmap `[['P','P'],['M','M'],['M','N']]` means that all residues with matching pbbnums in chain P will be aligned, as well as residues in chain M in the first structure with residues in chains M and N in the second structure. You would use this chainmap if structure 1 is MHC class I, structure 2 is MHC class II, and you want to superimpose on all peptide and MHC residues.

In [30]:
#load pdbs
filename1='./data/experimental_structures/processed_updated/pdb_rotated/1ao7_0.pdb'
filename2='./data/experimental_structures/processed_updated/pdb_rotated/3mre_0.pdb'
structure1,_=pdb_tools.parse_pdb(filename1)
structure2,_=pdb_tools.parse_pdb(filename2)

#see what chains are present
structure1_dict=pdb_tools.get_structure_dict(structure1,include_coords=False)
structure2_dict=pdb_tools.get_structure_dict(structure2,include_coords=False)
print('chains in structure 1:',structure1_dict.keys())
print('chains in structure 2:',structure2_dict.keys())
#M, P, A, B are MHC class I, peptide, TCR chain 1 and TCR chain 2, respectively

#superimpose structure 1 onto structure 2 using chain map. Structure 1 is transformed in place.
#(For superimposing by resmap, use pdb_tools.superimpose_by_resmap)
chainmap=[['M','M']] #superimpose by MHC
pdb_tools.superimpose_by_chainmap(structure1,structure2,chainmap,CA_only=True) #outputs rmsd over residues used

chains in structure 1: dict_keys(['M', 'P', 'A', 'B'])
chains in structure 2: dict_keys(['M', 'P'])


0.6758569672360559

Compute RMSD of two structures. Do not forget to superimpose them first! Like superimposing, can be done using resmap or chainmap
- when using chainmap, will compute RMSD only over residues with matching pdbnums! Unpaired residues will be ignored. In the output, gives alpha carbon ('CA') and all-atom ('all') RMSDs. In all-atom, matches atoms with identical names. Note that if the two residues are different amino acids, the all-atom RMSD is computed over a subset of atoms that is not very meaningful, even though no error will be produced.

In [31]:
chainmap=[['P','P']] #compute RMSD between peptides
rmsd=pdb_tools.rmsd_by_chainmap(structure1,structure2,chainmap)
print(rmsd) 
#all-atom RMSD is not very meaningful, because the peptides in these two structures have different sequence

{'CA': 1.076512997489881, 'all': 1.6519973427841208}


Make contact maps for a structure. Takes a path to a pdb file and a path to an output directory. Pretty slow. 
-  two atoms are considered in contact if the distance between their centers is smaller than a threshold. I set the threshold to be 1.1 times the sum of VdW radii. The chosen values of the radii can be found in `pdb_tools.py`

In [34]:
#make contact maps
filename='./data/experimental_structures/processed_updated/pdb_rotated/1ao7_0.pdb'
output_dir='./tmp'
pdb_tools.make_contact_maps(filename,output_dir)

#read the result
with open(output_dir+'/1ao7_0.pckl','rb') as f:
    contact_maps=pickle.load(f)
    
#the output is a dict indexed by pairs of chain names
print(contact_maps.keys())
#each value is a dict indexed by pairs of chain+resnum
contact_map_peptide_mhc=contact_maps['M','P']
print('number of contacts between residues M1009 and P3:',contact_map_peptide_mhc['M1009 ', 'P   3 '])

processing 1ao7_0...
dict_keys([('M', 'P'), ('M', 'A'), ('M', 'B'), ('P', 'A'), ('P', 'B'), ('A', 'B')])
number of contacts between residues M1009 and P3: 4


## The Structure Database

### Description

The database is stored in `./data/experimental_structures/processed_updated`. 

In [2]:
database='./data/experimental_structures/processed_updated'

- A structure is a single biological assembly from a source RCSB pdb file. It can be a pMHC, a single-chain or double-chain TCR, or a pMHC-TCR complex. (For complexes, the TCR always has two chains.) A single RCSB pdb file contains one asymmetric unit which often has more than one biological assembly, and thus is split into multiple structures in the database. A structure is named e.g. `1a9b_0`, `1a9b_1`, where `1a9b` is the PDB id and `_0`, `_1`, ... index biological assemblies that come from this pdb file.
- For a structure, e.g. `1a9b_0`, there is a processed pdb file `$database/pdb/1a9b_0.pdb`, a summary of protein chains `$database/proteins/1a9b_0.pckl`, and a summary of info from the pdb source `$database/pdb_info/1a9b.pckl` (one per RCSB pdb file). 
- For structures that include pMHC, there is also a rotated pdb file in `$database/pdb_rotated`, which is obtained by superimposing the structure onto a reference structure by MHC residues. (The reference structures are stored in `./data/experimental_structures/ref_structures`, one per MHC class. The two reference structures were not superimposed.)
- A processed pdb file has up to five chains: P, M, N, A, B. Chain P is the peptide, chain M is MHC alpha-chain, chain N is MHC beta-chain (for class II), chains A and B are the TCR. (For an alpha-beta TCR, A and B correspond to alpha and beta. For single-chain or G/D, there is no canonical correspondence.) The MHC and TCR chains are numbered in IMGT numbering, as defined in seq_tools. The peptide is numbered in a canonical way by superimposing onto the reference structures.  More details on peptide numbering can be found in the slides for my talk.
- For each structure, there is also an mmcif file in `$database/mmcif`. In this mmcif file, all chains are joined in the order `PMNAB` into a single chain named `A`, so that it can be fed into the monomer AF pipeline.
- For each structure, there is also a file with interchain residue-by-residue contact counts in `$database/contact_maps`. See `pdb_tools` above for more details

In [3]:
##### a processed pdb file ##### 

#load a processed file
import pdb_tools
filename=database+'/pdb_rotated/6p2f_0.pdb'
structure,_=pdb_tools.parse_pdb(filename)

#parse the chain sequences
sequences=pdb_tools.get_chain_sequences(structure)
#get NUMSEQ object for the peptide sequence
pepseq=sequences['canonical']['P']
#look at sequence and numbering
print(pepseq.seq())
print(pepseq.data['pdbnum']) #the peptide in 6p2f has an 8-aa binding core, despite being 9-aa long

AAAKKGYCL
['   09' '   1 ' '   2 ' '   3 ' '   4 ' '   5 ' '   7 ' '   8 ' '   9 ']


In [4]:
##### a summary of protein chains ##### 

#load protein chain info for a structure
with open(database+'/proteins/6p2f_0.pckl','rb') as f:
    proteins=pickle.load(f)
#it is a dict with chain letters as keys;
#for each chain, the value is a dict with 'data' and 'info' values, 
#which we can use directly or turn into seq_tools.NUMSEQ
print(proteins.keys()) 
print(proteins['P'].keys())
import seq_tools
pep_obj=seq_tools.load_NUMSEQ(proteins['P']) #turn into NUMSEQ

#peptide sequence
print(pep_obj.seq())

dict_keys(['P', 'M'])
dict_keys(['data', 'info'])
rxrarAAAKKGYCL


In a peptide sequence, small letters indicate gaps, i.e. residues that are present in SEQRES but not in ATOM/HETATM in the pdb file. 'X' or 'x' stand for a hetero residue, such as citrulline

The .info dict for a peptide contains, besides the usual stuff:
- a tuple 'linker' of booleans: whether the peptide was cut from a longer chain on the left or on the right;
- 'hetero_res'    : a dict of 3-letter names of hetero-residues contained in the peptide. E.g. in 6p2f,                      there is an N-methyl-L-alanine in the peptide position 0.5. This information is taken from SEQRES records in pdb
- 'hetero'        : whether the peptide contains hetero residues. Will be set to False if those residues                     are missing in the ATOM/HETATM records in the pdb file, even if present in SEQRES

In [5]:
#the .info attribute for a peptide
print(pep_obj.info)

{'chain': 'P', 'gaps': array([],
      dtype=[('seq', '<U3'), ('num', '<i2'), ('ins', '<U1'), ('pdbnum', '<U5'), ('ss', '<U15'), ('mutations', '<U3')]), 'linker': [False, False], 'hetero': False, 'hetero_res': {'   05': 'MAA'}}


For MHC, the info contains class, species, locus, allele information, as well as gaps and mutations relative 
to the parent allele. Unlike the peptide, here the gaps are not included as small letters in the sequence,
but are kept in `.info['gaps']`. The first few residues of an MHC are missing very often.
<br>Note: sometimes the MHC allele is missing in the sequence database, in which case the parent allele 
may be identified incorrectly. In such case, the number of mutations in `.info['mutations']` may be high.

In [6]:
#MHC
print('--MHC--')
mhc_obj=seq_tools.load_NUMSEQ(proteins['M'])
print(mhc_obj.info)

--MHC--
{'species': '9606', 'class': 'I', 'chain': 'A', 'locus': 'B', 'allele': '08:222', 'gaps': array([],
      dtype=[('seq', '<U3'), ('num', '<i2'), ('ins', '<U1'), ('pdbnum', '<U5'), ('ss', '<U15'), ('mutations', '<U3')]), 'hetero_res': {}}


Note: since MHC alleles are assigned automatically, often instead of a familiar allele, e.g. `A*02:01` or `B*08:01`, something less familiar is assigned, e.g. `A*02:665` or `B*08:222`. This happens when the two alleles have the same sequence in the antigen-binding region. (Often they differ in the constant region, e.g. by a gap, so that a familiar allele sequence with some missing residues is closer to some more rare allele.) These are essentially the same alleles.

In [87]:
##### pdb info #####
with open(database+'/pdb_info/6p2f.pckl','rb') as f:
    pdb_info=pickle.load(f)
print(pdb_info)

{'deposition_date': '2019-05-21', 'resolution': 1.48, 'includes_mhc': True, 'includes_tcr': False}


`pdb_info` contains deposition date, resolution (which is missing in one or two structures), and flags for whether the structure contains an MHC or a TCR. Note: these flags are set to True if the BLAST search finds a sufficiently good hit for any chain in the source pdb w.r.t. the MHC or TCR database. The MHC or TCR may still not be included in the processed file if there were some errors. Most often, this happens with an MHC which is something like CD1 or MR1 presenting a non-peptidic antigen. Such MHC sequences are currently not included in my sequence database and therefore do not appear in processed pdb files, even though BLAST identifies them as MHC.

The purpose of the flags 'includes_mhc' and 'includes_tcr' is to distinguish bound vs unbound TCRs/MHCs, respectively.

### Summaries and Clusters

A summary of information about structures can be found in `$database/summary.pckl`. It is a dict indexed by structure names. The entry for each structure contains information from `pdb_info`, as well as entries indexed by chain letters 'P','M''N','A','B' (whichever are present) with the same information as in `proteins`. For entries that contain pMHC, TCR and/or a pMHC-TCR complex, there are also entries `'pmhc_id', 'tcr_A_id', 'tcr_B_id', 'complex_id'` respectively, that provide ids for the corresponding sequences. See the next paragraph for the lists that these ids reference to.

In [12]:
with open(database+'/summary.pckl','rb') as f:
    summary=pickle.load(f)
print(len(summary))
print(summary['1ao7_0'].keys())

2300
dict_keys(['pdbid', 'P', 'M', 'A', 'B', 'pep_hetero', 'pep_gaps', 'includes_mhc', 'includes_tcr', 'date', 'resolution', 'pmhc_id', 'tcr_A_id', 'tcr_B_id', 'complex_id'])


Often there are multiple structures for the same sequence. (Mostly these are different biological assemblies from the same RCSB pdb file, but can also be from different pdb files.) We aggregate structures by sequence in `$database/pmhcs.pckl` for pMHCs, in `$database/tcrs.pckl` for TCR chains, and in `$database/complexes.pckl` for complexes. 
-  each of these files contains a python list, where each entry corresponds to a unique sequence and possibly multiple structures. Each entry is a dict
-  a single structure will appear twice in `tcrs.pckl` if it contains both chains of a TCR
-  a single structure will appear in all three lists if it contains a pMHC-TCR complex
-  the structures for an entry `x` are collected in `x['pdbs']`; the value `x['pdbs']` is a list of tuples. Each tuple contains the pdbid and a dict of basic properties of the corresponding structure, including resolution. The structures in the list `x['pdbs']` were sorted by the number of missing residues and by resolution. Thus, you can assume that the first element in `x['pdbs']` is "the best" structure for a given sequence
-  the sequences themselves are stored in the form of {'data':data,'info':info} dicts under the keys `'P','M','N'` for pMHCs, `'obj'` for TCR chains, and `'P','M','N','A','B'` for complexes
-  Note! Missing residues in the sequences here have been restored using aligned reference alleles for MHCs and TCRs, or using SEQRES for the peptides. (This is unlike in `proteins` or `summary.pckl`.) In particular, two structures whose sequences  differ only in what residues are missing will be merged into the same entry in the list
-  each entry `x` has a `x['date']` value. This is the earliest date over all pdb files in this entry
-  each entry `x` has `x['id']` value. This is the id of the sequence, which is simply the index of this entry in the corresponding list
-  for pMHCs and TCRs, `x['bound']` can take values `'b','u','bu'`, which means that the structures within this entry contain this entity only in bound, only in unbound, or both in bound and unbound form.
-  for pMHCs and complexes, `x['class']` is the MHC class
-  for pMHCs, `x['pep_pdbnum']` is a string with pdbnums for peptide numbering. pMHCs with the same sequence but different pep_pdbnum (and hence different geometry) are assigned to different entries!
-  entries with `cluster_seq` in the key are ids of the cluster that the sequence belongs to. The clustering is described below
-  for pMHCs, `x['cluster_CA']` is the id of the cluster in clustering by peptide alpha carbon positions, as described below

In [3]:
print('pMHCs:')
with open(database+'/pmhcs.pckl','rb') as f:
    pmhcs=pickle.load(f)
print(len(pmhcs))
print(pmhcs[0].keys())
print(pmhcs[0]['pdbs'])
print()

print('TCR chains:')
with open(database+'/tcrs.pckl','rb') as f:
    tcrs=pickle.load(f)
print(len(tcrs))
print(tcrs[0].keys())
print()

print('pMHC-TCR complexes:')
with open(database+'/complexes.pckl','rb') as f:
    complexes=pickle.load(f)
print(len(complexes))
print(complexes[0].keys())
print()

pMHCs:
896
dict_keys(['P', 'M', 'class', 'pdbs', 'date', 'pep_pdbnum', 'id', 'bound', 'cluster_seq', 'cluster_CA'])
[('2hn7_0', {'pep_gaps': 0, 'linker': False, 'resolution': 1.6, 'includes_tcr': 0})]

TCR chains:
455
dict_keys(['chain', 'obj', 'pdbs', 'date', 'id', 'bound', 'cluster_seq'])

pMHC-TCR complexes:
234
dict_keys(['P', 'M', 'N', 'A', 'B', 'pmhc_id', 'tcr_A_id', 'tcr_B_id', 'pdbs', 'date', 'class', 'id', 'pmhc_cluster_seq', 'tcr_A_cluster_seq', 'tcr_B_cluster_seq', 'complex_cluster_seq'])



pMHCs and TCR chains were clustered by sequence similarity. (Hierarchical clustering by edit distance, cutoff 4, method `'single'`.) The lists of resulting clusters are stored in the lists `$database/pmhcs_seq_clusters.pckl` and `$database/tcrs_seq_clusters.pckl`. The clusters for pMHC-TCR complexes were formed by merging entities with the same clusters for pMHC and both TCR chains, with the results stored in the list `$database/complex_seq_clusters.pckl`
-  note that there are many examples where two or more complexes have similar TCR but different pMHCs, or similar pMHCs but different TCRs. Those complexes were not merged in clustering.
-  each entry in cluster lists has elements:
    - `id`, which is simply the index in the list
    - `{pmhc/tcr/complex}_id`, which is the list of ids of sequences forming the cluster (i.e. indices in the list `pmhcs.pckl/tcrs.pckl/complexes.pckl`.)
    - `pdbs`, which is a list of all structures withing the cluster, sorted by the number of missing residues and resolution
    - `date`, which is the deposition date of the oldest pdb within the cluster
- for pMHCs, there is also a list `$database/pmhcs_CA_clusters.pckl`, which are clusters by peptide alpha-carbon positions. The clusters were formed by k-means, with each vector initialized at a representative from `pmhcs_seq_clusters`

In [8]:
print('pMHC sequence clusters:')
with open(database+'/pmhcs_seq_clusters.pckl','rb') as f:
    pmhcs_seq_clusters=pickle.load(f)
print(len(pmhcs_seq_clusters))
print(pmhcs_seq_clusters[0].keys())
print()

print('pMHC CA clusters:')
with open(database+'/pmhcs_CA_clusters.pckl','rb') as f:
    pmhcs_CA_clusters=pickle.load(f)
print(len(pmhcs_CA_clusters))
print(pmhcs_CA_clusters[0].keys())
print()

print('TCR chain sequence clusters:')
with open(database+'/tcrs_seq_clusters.pckl','rb') as f:
    tcrs_seq_clusters=pickle.load(f)
print(len(tcrs_seq_clusters))
print(tcrs_seq_clusters[0].keys())
print() 

print('pMHC-TCR sequence clusters:')
with open(database+'/complex_seq_clusters.pckl','rb') as f:
    complex_seq_clusters=pickle.load(f)
print(len(complex_seq_clusters))
print(complex_seq_clusters[0].keys())

pMHC sequence clusters:
453
dict_keys(['id', 'pmhc_ids', 'pdbs', 'date'])

pMHC CA clusters:
453
dict_keys(['id', 'pmhc_ids', 'pdbs', 'date'])

TCR chain sequence clusters:
375
dict_keys(['id', 'tcr_ids', 'pdbs', 'date'])

pMHC-TCR sequence clusters:
159
dict_keys(['id', 'complex_ids', 'pdbs', 'date'])


Database usage example: peptide-MHC pairs that can adopt multiple binding registers

In [18]:
#build pmhc sequence to id dict, to find sequences that appear in multiple entries
seq_to_id={} #seq: [id,id..]
for x in pmhcs:
    seq=[''.join(x['P']['data']['seq'])]
    seq+=''.join(x['M']['data']['seq'])
    if 'N' in x:
        seq+=''.join(x['N']['data']['seq'])
    seq='|'.join(seq)
    seq_to_id.setdefault(seq,[]).append(x['id'])
#print pepseq, MHC allele, peptide numberings and pdb ids for pMHCs with multiple binding registers
for v in seq_to_id.values():
    if len(v)>1:
        pmhc=pmhcs[v[0]]
        pepseq=''.join(pmhc['P']['data']['seq'])
        mhc_allele='*'.join([pmhc['M']['info']['locus'],pmhc['M']['info']['allele']])
        if 'N' in pmhc:
            mhc_allele+='/'+'*'.join([pmhc['N']['info']['locus'],pmhc['N']['info']['allele']])
        print('{:15s} {:20s}'.format(pepseq,mhc_allele))
        for u in v:
            pdbnum=pmhcs[u]['pep_pdbnum']
            pdb_ids=[a[0] for a in pmhcs[u]['pdbs']]
            print('{:25s} {}'.format(pdbnum.replace(' ',''),pdb_ids))
        print()

AAGIGILTV       A*02:665            
2,3,4,5,51,6,7,8,9        ['6d78_0', '6eqb_0', '6eqa_0']
1,2,3,4,5,6,7,8,9         ['3qfd_0', '3qfd_1', '2guo_0', '2guo_1', '3qdj_0', '3qeq_0']

ELAXXLTV        A*02:665            
1,2,3,4,5,7,8,9           ['3o3b_0', '3o3b_1']
1,2,3,4,5,7,8,9           ['3o3d_0', '3o3d_1']

LYLVCGERGF      K*d                 
1,2,3,4,5,51,6,7,8,9      ['4z78_0']
1,2,3,4,5,6,7,8,9,10      ['4z78_1']

MMWDRGLGMM      A*02:665            
1,2,3,4,5,51,6,7,8,9      ['6amt_0', '6amt_1']
1,2,3,4,5,6,7,8,9,10      ['6amu_0']

KAFSPEVIPMF     B*57:03             
1,2,3,4,5,51,59,6,7,8,9   ['2bvo_0']
1,2,3,4,5,7,8,9,10,11,12  ['2bvq_0']

XLAXXLTV        A*02:665            
1,2,3,4,5,7,8,9           ['3o3a_0', '3o3a_1']
1,2,3,4,5,7,8,9           ['3o3e_0', '3o3e_1']

LLFGXPVYV       A*02:665            
1,2,3,4,5,6,7,8,9         ['3d3v_0']
1,2,3,4,5,6,7,8,9         ['3d39_0']



### How to Create/Update

To create the database, you need to use MHC and TCR MSAs from `./data/msas` to search the PDB database using `tfold_MSA_tools.py`. Then you need to run `process_pdb.py` on each pdb file, and afterwards aggregate and cluster the data. 
-  these operations are run from the notebook `create_structure_database.ipynb`. The code there is pretty straightforward, but currently is not well-annotated
-  the main workhorse is the tool `process_pdbs.py` that parses a pdb file
-  the process is rather slow. To process the thousands of pdb files found by RCSB searching, it is recommended to run processing on a cluster

Here is an example of processing a pdb file. The output directory has the same structure and contains the same kind of data as `$database`.

In [25]:
pdb_id='1ao7'

import process_pdbs
import time
pdb_source_dir='./data/experimental_structures/pdb_source'
input_file=pdb_source_dir+f'/{pdb_id}.pdb'
output_dir='./tmp/processed'

t0=time.time()
process_pdbs.process_structure(input_file,output_dir)
print('completed in {:5.1f} s'.format(time.time()-t0))

print('what is in the output dir:')
print(os.listdir(output_dir))

processing structure 1ao7...
completed in  10.4 s
what is in the output dir:
['pdb', 'proteins', 'pdb_info', 'debug']
